# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%reload_ext dotenv
%dotenv
import sys
sys.path.append(os.getenv('SRC_DIR'))
from logger import get_logger
_logs = get_logger(__name__)

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

c:\Users\shaum\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
price_data_dir = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive=True)
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
parquet_files

['../../05_src/data/prices\\AAPL\\AAPL_2000\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2000\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.1.parquet',
 '../../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [21]:
# Write your code below.

dd_shift = dd_px.groupby("Ticker", group_keys = False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)

dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/ x['Close_lag_1'] - 1
)

dd_feat = dd_rets.assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
)

#dd_feat = dd_rets.assign(
 #   hi_lo_range=lambda x: x["High"] - x["Low"]

#dd_feat = dd_shift + dd_rets + dd_hi_lo_range

dd_feat.compute()

C:\Users\shaum\AppData\Local\Temp\ipykernel_2564\3465474622.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby("Ticker", group_keys = False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Returns,hi_lo_range
Ticker,,,,,,,,,,,
AAPL,2000-01-03,0.843077,0.999442,1.004464,0.907924,0.936384,535796800.0,2000,NaN,NaN,0.096540
AAPL,2000-01-04,0.771997,0.915179,0.987723,0.903460,0.966518,512377600.0,2000,0.999442,-0.084310,0.084263
AAPL,2000-01-05,0.783293,0.928571,0.987165,0.919643,0.926339,778321600.0,2000,0.915179,0.014633,0.067522
AAPL,2000-01-06,0.715509,0.848214,0.955357,0.848214,0.947545,767972800.0,2000,0.928571,-0.086538,0.107143
AAPL,2000-01-07,0.749401,0.888393,0.901786,0.852679,0.861607,460734400.0,2000,0.848214,0.047369,0.049107
...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,2025-01-17,405.709991,405.709991,407.290009,402.290009,406.040009,270600.0,2025,402.720001,0.007424,5.000000
ZBRA,2025-01-21,418.070007,418.070007,419.850006,407.619995,407.619995,446000.0,2025,405.709991,0.030465,12.230011
ZBRA,2025-01-22,420.570007,420.570007,427.760010,419.589996,425.239990,497500.0,2025,418.070007,0.005980,8.170013


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.
#dd_px = dd.read_parquet(parquet_files)

#pd_df = dd_px.compute()

pd_feat = dd_feat.compute()

pd_feat = dd_feat.assign(
    moving_mean_ret = lambda x: x['Returns'].rolling(10).mean()
)


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? No it was not necessary to convert to pandas to calculate the moving mean calculation as Dask can support that calculation.
+ Would it have been better to do it in Dask? Why? It would have been better to do it in Dask when it is a large dataset, in this case, it is not a huge dataset so eith is fine.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.